# Julia 機器學習：GLM 線性迴歸

## 作業 027：波士頓房價預測資料集

請使用 GLM 中的模型，建立一個預測模型來預測波士頓的房價。

In [2]:
using GLM, RDatasets, MLDataUtils, DataFrames

## 讀取資料

In [17]:
boston = dataset("MASS", "Boston")
first(boston, 10)

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311
8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311
9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311


In [18]:
boston[!, :LogMedV] = log.(boston[!, :MedV])

506-element Array{Float64,1}:
 3.1780538303479458
 3.0726933146901194
 3.5467396869528134
 3.5085558999826545
 3.5890591188317256
 3.3568971227655755
 3.131136910560194
 3.299533727885655
 2.803360380906535
 2.9391619220655967
 2.70805020110221
 2.9391619220655967
 3.077312260546414
 ⋮
 3.1986731175506815
 3.139832617527748
 2.9806186357439426
 2.9069010598473755
 3.054001181677967
 2.8622008809294686
 2.8213788864092133
 3.109060958860994
 3.0252910757955354
 3.173878458937465
 3.091042453358316
 2.4765384001174837

In [19]:
indecies = MLDataUtils.shuffleobs(collect(1:nrow(boston)))
train_ind, test_ind = MLDataUtils.splitobs(indecies, at = 0.7)

train = boston[train_ind, :]
test = boston[test_ind, :]


,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.49298,0.0,9.9,0,0.544,6.635,82.5,3.3175,4,304
2,13.0751,0.0,18.1,0,0.58,5.713,56.7,2.8237,24,666
3,24.3938,0.0,18.1,0,0.7,4.652,100.0,1.4672,24,666
4,0.09849,0.0,25.65,0,0.581,5.879,95.8,2.0063,2,188
5,0.06047,0.0,2.46,0,0.488,6.153,68.8,3.2797,3,193
6,8.71675,0.0,18.1,0,0.693,6.471,98.8,1.7257,24,666
7,0.03537,34.0,6.09,0,0.433,6.59,40.4,5.4917,7,329
8,1.51902,0.0,19.58,1,0.605,8.375,93.9,2.162,5,403
9,9.32909,0.0,18.1,0,0.713,6.185,98.7,2.2616,24,666


In [20]:
model = lm(@formula(LogMedV ~ Crim + Chas + NOx + Rm + Dis + Rad + Tax + PTRatio + Black + LStat), train)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

LogMedV ~ 1 + Crim + Chas + NOx + Rm + Dis + Rad + Tax + PTRatio + Black + LStat

Coefficients:
────────────────────────────────────────────────────────────────────────────────────
                    Coef.   Std. Error       t  Pr(>|t|)     Lower 95%     Upper 95%
────────────────────────────────────────────────────────────────────────────────────
(Intercept)   4.21021      0.240817      17.48    <1e-48   3.73654       4.68387
Crim         -0.0130216    0.00169648    -7.68    <1e-12  -0.0163584    -0.00968478
Chas          0.0920605    0.040989       2.25    0.0253   0.011439      0.172682
NOx          -0.515412     0.17793       -2.90    0.0040  -0.865384     -0.165441
Rm            0.060532     0.0192787      3.14    0.0018   0.0226127     0.0984514
Dis          -0.0421987    0.00779493    -5.41    <1e-6   -0.057530

In [23]:
exp.(predict(model, test))

152-element Array{Float64,1}:
 29.10031929020686
 16.864968689819033
  8.895493810621382
 19.167374352890256
 23.72552449428145
 17.119649266668464
 27.037864392169368
 41.28280170553944
 15.677134898691829
 17.207624850631078
 26.047490055393304
 18.61024769609051
 22.727753136037983
  ⋮
 22.91747921023237
 19.360438681343073
 20.660188549848883
 27.178024069529076
 13.005797660158906
 36.97481705069093
 14.621601331765792
 30.44367873404925
 27.214775532832906
 28.826500161144597
 35.53460274433197
 24.60991255988824

In [28]:
adjr²(model)

0.7864562349089663